# Opinion Formation

### Imports

In [1]:
import numpy as np
import random
import math
import os
import scipy as sp
import cellpylib as cpl
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation